In [2]:
import torch
from transformers import BertTokenizerFast, GPT2LMHeadModel

In [3]:
# 設定裝置為 GPU 或 CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# 載入訓練好的模型和 tokenizer
model_path = "./NetflixGPT-chinese"  # 修改為你訓練模型的儲存路徑
tokenizer = BertTokenizerFast.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(21131, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=21131, bias=False)
)

In [5]:
# 定義 inference 測試函數
def generate_description(title):
    input_text = f"標題: {title} 描述:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)
    
    # 使用模型進行生成
    output = model.generate(input_ids, max_length=512, num_return_sequences=1, no_repeat_ngram_size=2, 
                            pad_token_id=tokenizer.eos_token_id, early_stopping=True)
    
    # 解碼生成的描述
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_text = ''.join(generated_text.split(' '))
    new_input_text = ''.join(input_text.split(' '))
    return generated_text.replace(new_input_text, "").strip()

In [6]:
# 測試生成效果
test_titles = ["精神病特工", "追星女孩", "非法女人"]
for title in test_titles:
    print(f"Title: {title}")
    print("Generated Description:", generate_description(title))
    print("-" * 50)

Title: 精神病特工


c:\Users\swguo\anaconda3\lib\site-packages\transformers\generation\configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Generated Description: 特·格里爾斯和他的朋友們在一個小鎮上度過了一年的假期，他們的生活在他最好的時刻裡面臨著一些令人毛骨悚然的事情。
--------------------------------------------------
Title: 追星女孩
Generated Description: 在一個小鎮上，一位年輕的女子在她的家鄉度過了一年的假期，她在那裡遇到了兩個女人，他們都在尋找自己的方法，並在這個時候遇見了他。
--------------------------------------------------
Title: 非法女人
Generated Description: 在一個被一位女性拒絕的城市裡，一名女子在她的家裡被她所愛的女孩綁架，她在那裡遇到了一隻神秘的貓，並在這個女兒的生活被打破。
--------------------------------------------------


In [7]:
# 定義生成描述的函數（使用 top_k 和 temperature）
def generate_description(title, max_length=512, top_k=50, temperature=0.7):
    # 構建輸入文本，不添加特殊標記
    input_text = f"標題: {title} 描述:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    
    # 使用模型進行生成，設置 top_k 和 temperature
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length + len(input_ids[0]),  # 增加總長度限制
            do_sample=True,                            # 啟用隨機抽樣
            top_k=top_k,                               # 設置 top_k
            temperature=temperature,                   # 設置 temperature
            no_repeat_ngram_size=2,                    # 防止重複 n-grams
            pad_token_id=tokenizer.pad_token_id
        )
    
    # 只提取生成的描述部分
    generated_text = tokenizer.decode(output[0][input_ids.shape[1]:], skip_special_tokens=True)
    generated_text = ''.join(generated_text.split(' '))
    new_input_text = ''.join(input_text.split(' '))
    return generated_text.replace(new_input_text, "").strip()

In [8]:
# 測試生成效果
test_titles = ["精神病特工", "追星女孩", "非法女人"]
for title in test_titles:
    print(f"Title: {title}")
    print("Generated Description:", generate_description(title))
    print("-" * 50)

Title: 精神病特工
Generated Description: 一位精明的醫生和一些有抱負的前疾病的單身同事，他們在與一個被診斷出異常後患有精子狀態的人合作，並建立了聯繫。
--------------------------------------------------
Title: 追星女孩
Generated Description: hercleman和她的朋友們在一個受到她媽媽的阻礙的星球上穿梭，共同尋找失蹤的女兒。
--------------------------------------------------
Title: 非法女人
Generated Description: 在她的婚姻關係中陷入僵局，一名法律系學生在一場意外中失蹤後，開始了一段新的關於生活，她和一位英俊的女性之間的愛情也開花了。
--------------------------------------------------
